In [ ]:
import tensorflow as tf
from keras.layers import Dense, Flatten, Lambda, Activation, MaxPooling2D,SpatialDropout2D,Dropout
from keras.layers.convolutional import Convolution2D
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import json
import DataProvider

tf.python.control_flow_ops = tf

number_of_epochs = 10
number_of_samples_per_epoch = 20032
number_of_validation_samples = 6400
learning_rate = 1e-4
activation_relu = 'relu'

# Our model is based on NVIDIA's "End to End Learning for Self-Driving Cars" paper
# Source:  https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf
class BehavioralCloningModel():
    def __init__(self):
        self.model = self.__nvidia_like()
        self.history = None
        
    def __nvidia_like(self):
        model = Sequential()
        model.add(Lambda(lambda x: x / 127.5 - 1.0, input_shape=(64, 64, 3)))
        # starts with five convolutional and maxpooling layers
        model.add(Convolution2D(24, 5, 5, border_mode='same', subsample=(2, 2)))
        model.add(Activation(activation_relu))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        model.add(SpatialDropout2D(0.2))

        model.add(Convolution2D(36, 5, 5, border_mode='same', subsample=(2, 2)))
        model.add(Activation(activation_relu))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        model.add(SpatialDropout2D(0.2))

        model.add(Convolution2D(48, 5, 5, border_mode='same', subsample=(2, 2)))
        model.add(Activation(activation_relu))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        model.add(SpatialDropout2D(0.2))

        model.add(Convolution2D(64, 3, 3, border_mode='same', subsample=(1, 1)))
        model.add(Activation(activation_relu))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        model.add(SpatialDropout2D(0.2))

        model.add(Convolution2D(64, 3, 3, border_mode='same', subsample=(1, 1)))
        model.add(Activation(activation_relu))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
        model.add(SpatialDropout2D(0.2))

        model.add(Flatten())

        # Next, five fully connected layers
        model.add(Dense(1164))
        model.add(Activation(activation_relu))
        model.add(Dropout(0.5))
        
        model.add(Dense(100))
        model.add(Activation(activation_relu))

        model.add(Dense(50))
        model.add(Activation(activation_relu))

        model.add(Dense(10))
        model.add(Activation(activation_relu))
        model.add(Dropout(0.5))
        
        model.add(Dense(1))

        model.summary()
        model.compile(optimizer=Adam(learning_rate), loss="mse", )
        return model
        
    def execute(self,train,valid, nspe=number_of_samples_per_epoch,nvspe=number_of_validation_samples):
        self.history = self.model.fit_generator(train,
                              samples_per_epoch=number_of_samples_per_epoch,
                              nb_epoch=number_of_epochs,
                              validation_data=valid,
                              nb_val_samples=number_of_validation_samples,
                              verbose=1)
        
    
    def save(self, mname='model.json', wname='model.h5'):
        jsmodel = self.model.to_json()
        with open(mname, 'w') as jsfile:
            json.dump(jsmodel, jsfile)

        #self.model.save_weights(wname)
        self.model.save(wname)
        print("Model Saved in Json and h5 formats")
        
    def history(self):
        ### print the keys contained in the history object
        print(self.history.history.keys())

        ### plot the training and validation loss for each epoch
        plt.plot(self.history.history['loss'])
        plt.plot(self.history.history['val_loss'])
        plt.title('model mean squared error loss')
        plt.ylabel('mean squared error loss')
        plt.xlabel('epoch')
        plt.legend(['training set', 'validation set'], loc='upper right')
        plt.show()     

# create two generators for training and validation
dp = DataProvider.DataProvider()
train = dp.getbatch(64,'train')
valid = dp.getbatch(64,'valid')
bcm = BehavioralCloningModel()
bcm.execute(train,valid,dp.train_length, dp.validation_length)
bcm.save()
bcm.history()